# Main Jupyter notebook file that prepares and standardizes data in advance of the main calculation and score


## will require the key steps.
### Main car data records:

1.) combining all car manufacturers into the same database
2.) highlighting null values
3.) standardizing formatting accross columns in centralized 'used car data' csv
4.) standardizing datatype for cost to be float

### preparing maintenance costs to match car data

1.) renaming columns

data cleaning and formatting
merging maintenance costs into central DB


### preparing ratings to match car data
data cleaning and formatting
merging car ratings into central DB

In [1]:
# key imports
import os
import csv
import glob
import pandas as pd
import datetime

In [2]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [3]:
from fuzzywuzzy import fuzz, process

# Combining all CSV files into a central CSV for processing

##


In [4]:
# Define the path where your CSV files are located
pathToExistingCSVs = './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/*.csv'


# Define the output file where you want to combine all CSVs
output_file = './Datasets/Processed_Data/used_car_data_combined.csv'

# Ensure output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Clear the output file content before writing
open(output_file, 'w').close()

# Get a list of all CSV files in the directory, in alphabetical order based on brand
csv_files = glob.glob(pathToExistingCSVs)
csv_files = sorted(csv_files, key=lambda x: x.split('/')[-1])

print(csv_files)

# Initialize a variable to store all CSV headers (to ensure they are the same across files, plus the new "brand" column)
headers = []

['./Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/audi.csv', './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/bmw.csv', './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/ford.csv', './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/hyundai.csv', './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/merc.csv', './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/skoda.csv', './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/toyota.csv', './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/vauxhall.csv', './Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/vw.csv']


# Correcting mismatched header for Hyundai

In [5]:
hyundaiDF = pd.read_csv('./Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/hyundai.csv')
hyundaiDF.rename(columns={'tax(£)': 'tax'}, inplace=True)
hyundaiDF.to_csv('./Datasets/Provided_Datasets/used-car-dataset-challenge/Used_Car_Data_(incl mpg)/hyundai.csv', index=False, header=True)

In [6]:
# Open the output file in write mode
with open(output_file, 'w', newline='', encoding='utf-8') as f_out:
    writer = csv.writer(f_out)

    # Process each file in turn
    for file in csv_files:
        try:
            # Extract brand name from the file name
            brand_name = os.path.basename(file).replace('.csv', '')

            with open(file, newline='', encoding='utf-8') as f_in:
                reader = csv.reader(f_in)
                header = next(reader)  # Read the header row

                # If it's the first file, add "brand" to the headers list and write the header
                if not headers:
                    headers = ['brand'] + header
                    writer.writerow(headers)
                else:
                    # catch here to make sure all files have similar structure
                    assert headers[1:] == header, f"Headers have a mismatch in the manufacturer: {file} with {header}"


                # Write the data rows with the "brand" column
                for row in reader:
                    writer.writerow([brand_name] + row)
        except Exception as e:
            print(f"Theres an error with {file}: {e}")

            
print(f"All CSV files have been combined into {output_file}")
print(f"all unique headers are: {headers}")

All CSV files have been combined into ./Datasets/Processed_Data/used_car_data_combined.csv
all unique headers are: ['brand', 'model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize']


# Null value check

In [7]:
df = pd.read_csv('./Datasets/Processed_Data/used_car_data_combined.csv')

collective_null_values = df.isnull().sum().sum()
print(f"There are {collective_null_values} null entries present in combined set.")

df.head()

There are 0 null entries present in combined set.


,brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,audi,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,audi,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,audi,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,audi,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,audi,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


# Abnormal values check

checking to verify any incorrect classifications

In [8]:
# Correcting Mercedes name
unique_brands = df['brand'].unique()
print(unique_brands)

df['brand'] = df['brand'].replace(r'merc', 'mercedes', regex=True)

unique_brands = df['brand'].unique()
print(unique_brands)

# Transmission class checks
unique_transmissions = df['transmission'].unique()
unique_transmissionsOtherCount = (df['transmission'] == 'Other').sum()
print(f"Unique transmissions: {unique_transmissions}.")
print(f"'Other count': {unique_transmissionsOtherCount}")
print(f"Dropping 'other' class in transmission due to input error\n\n")

cars_to_drop_transmission = df[df['transmission'] == 'Other'].index
df.drop(cars_to_drop_transmission, inplace=True)


##### Fuel class checks
unique_fuelType = df['fuelType'].unique()
unique_fuelTypeCount = (df['fuelType'] == 'Other').sum()
print(f"Unique fuel type: {unique_fuelType}.")
print(f"'Other' fuel count: {unique_fuelTypeCount}")

cars_to_drop_fuel = df[df['fuelType'] == 'Other'].index
df.drop(cars_to_drop_fuel, inplace=True)
print(f"Dropping irregular fuel types for these entries.\n\n")


###### year range check
current_unix_timestamp = datetime.datetime.now().timestamp()
current_datetime = datetime.datetime.fromtimestamp(current_unix_timestamp)
current_year = current_datetime.year


oldest_car_value = df['year'].min()
newest_car_value = df['year'].max()
print(f"Oldest car: {oldest_car_value}.")
print(f"Newest car: {newest_car_value}.")


print(f"Newest car entries that exceed current year and are likely entered in error, will drop from dataset")
cars_to_drop_year = df[df['year'] > current_year].index
over_current_year = (df['year'] > current_year).sum()

print(f"number of cars being dropped due to exceeding {current_year}: {over_current_year}.")
df.drop(cars_to_drop_year, inplace=True)

df['model'] = df['model'].str.lstrip()
df.to_csv('./Datasets/Processed_Data/used_car_data_combined.csv', index=False, header=True)

['audi' 'bmw' 'ford' 'hyundai' 'merc' 'skoda' 'toyota' 'vauxhall' 'vw']
['audi' 'bmw' 'ford' 'hyundai' 'mercedes' 'skoda' 'toyota' 'vauxhall' 'vw']
Unique transmissions: ['Manual' 'Automatic' 'Semi-Auto' 'Other'].
'Other count': 9
Dropping 'other' class in transmission due to input error


Unique fuel type: ['Petrol' 'Diesel' 'Hybrid' 'Other' 'Electric'].
'Other' fuel count: 246
Dropping irregular fuel types for these entries.


Oldest car: 1970.
Newest car: 2060.
Newest car entries that exceed current year and are likely entered in error, will drop from dataset
number of cars being dropped due to exceeding 2024: 1.


# Preparing Maintainance costs CSV in advance of merger.

In [9]:
maintenanceDF = pd.read_csv('./Datasets/Provided_Datasets/Used-car-dataset-challenge/Car_Maintenance_Costs.csv')
print(maintenanceDF.head())

     Make      Model  Year  MaintenanceCostYearly
0     BMW         x3  2017                 607.20
1   SKODA      karoq  2018                 359.07
2  toyota      Hilux  2015                 655.78
3    audi        RS4  2015                 859.98
4     bmw   5 SERIES  2004                1264.16


In [10]:
# Load the datasets
combined_cars_df = pd.read_csv(r'Datasets/Processed_Data/used_car_data_combined.csv')
maintenanceDF = pd.read_csv(r'Datasets/Provided_Datasets/used-car-dataset-challenge/Car_Maintenance_Costs.csv')


#print(combined_cars_df.head())
print(maintenanceDF.head())


# Convert 'Make' column to lowercase and replace 'hyundi' with 'hyundai'. 
maintenanceDF['Make'] = maintenanceDF['Make'].str.lower().replace(r'hyundi', 'hyundai', regex=True)

# Ensure 'MaintenanceCostYearly' is a float with two decimal places
maintenanceDF['MaintenanceCostYearly'] = maintenanceDF['MaintenanceCostYearly'].astype(float).round(2)

# Strip leading/trailing whitespace and convert 'Model' column to lowercase
maintenanceDF['Model'] = maintenanceDF['Model'].str.title()


# making all columns lowercase for future merger
maintenanceDF.rename(columns={'Make': 'Make'.lower()}, inplace=True)
maintenanceDF.rename(columns={'Model': 'Model'.lower()}, inplace=True)
maintenanceDF.rename(columns={'Year': 'Year'.lower()}, inplace=True)

print(maintenanceDF.head())
maintenanceDF.to_csv('./Datasets/Processed_Data/Car_Maintenance_Costs_Sanitized.csv', index=False, header=True)

     Make      Model  Year  MaintenanceCostYearly
0     BMW         x3  2017                 607.20
1   SKODA      karoq  2018                 359.07
2  toyota      Hilux  2015                 655.78
3    audi        RS4  2015                 859.98
4     bmw   5 SERIES  2004                1264.16
     make      model  year  MaintenanceCostYearly
0     bmw         X3  2017                 607.20
1   skoda      Karoq  2018                 359.07
2  toyota      Hilux  2015                 655.78
3    audi        Rs4  2015                 859.98
4     bmw   5 Series  2004                1264.16


# 'model' class prep.


In [11]:
maintenanceDF = pd.read_csv('./Datasets/Processed_Data/Car_Maintenance_Costs_Sanitized.csv')
#print(maintenanceDF.head())


# Function to get the best match for a given model
def get_best_match(model, choices):
    
    if (model.lower() == 'ka') or (model.lower() == 'Ka'):
        return ' KA'
    
    else:

        match = process.extractOne(model, choices, scorer=process.fuzz.token_sort_ratio, score_cutoff=75)
        # If a match is found with a score above the cutoff, return the matched model, else return the original
        return match[0] if match else model



# Strip leading/trailing whitespace and convert 'Model' column to lowercase
maintenanceDF['model'] = maintenanceDF['model'].str.lower()
maintenanceDF['model'] = maintenanceDF['model'].str.title()
maintenanceDF['model'] = maintenanceDF['model'].str.strip()


# Create a list of unique models from the combined_cars dataset for matching
model_choices = combined_cars_df['model'].unique()

# Replace 'Model' entries in df with the best match from combined_cars_df
maintenanceDF['model'] = maintenanceDF['model'].apply(lambda x: get_best_match(x, model_choices))
maintenanceDF['model'] = maintenanceDF['model'].str.lstrip()


maintenanceDF.rename(columns={'make': 'brand'}, inplace=True)
maintenanceDF.to_csv('./Datasets/Processed_Data/Car_Maintenance_Costs_Sanitized.csv', index=False, header=True)

In [12]:
unique_models_maintenance = maintenanceDF['model'].unique()
print(f'Maintenance DF has {len(unique_models_maintenance)} unique entries.')
#print(unique_models_maintenance)

unique_models_combined = combined_cars_df['model'].unique()
print(f'Combined dataset DF has {len(unique_models_combined)} unique entries.')
#print(unique_models_combined)


unique_models_existing_set = set(unique_models_combined)
unique_models_maintenance_set = set(unique_models_maintenance)

unique_to_existing_only = unique_models_existing_set - unique_models_maintenance_set
print("Entries only present in existing files: ")
print(unique_to_existing_only)

unique_to_maintenance_only = unique_models_maintenance_set - unique_models_existing_set
print("Entries only present in maintenance files: ")
print(unique_to_maintenance_only)

Maintenance DF has 195 unique entries.
Combined dataset DF has 195 unique entries.
Entries only present in existing files: 
set()
Entries only present in maintenance files: 
set()


# Once all car models in Maintenance set acceptable, can merge into main


In [13]:
main_car_data_DF = pd.read_csv('./Datasets/Processed_Data/used_car_data_combined.csv')
maintenance_DF = pd.read_csv('./Datasets/Processed_Data/Car_Maintenance_Costs_Sanitized.csv')

main_car_data_DF.head()


,brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,audi,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,audi,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,audi,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,audi,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,audi,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [14]:
maintenance_DF.head()

,brand,model,year,MaintenanceCostYearly
0,bmw,X3,2017,607.20
1,skoda,Karoq,2018,359.07
2,toyota,Hilux,2015,655.78
3,audi,RS4,2015,859.98
4,bmw,5 Series,2004,1264.16


In [15]:
combinedDF = pd.merge(main_car_data_DF, maintenance_DF[['brand', 'model', 'year', 'MaintenanceCostYearly']],
                     on=['brand', 'model', 'year'],
                     how='left')

#combinedDF.head()
combinedDF.to_csv('./Datasets/Processed_Data/used_car_data_combined.csv', index=False, header=True)

In [16]:
carReviewsDF = pd.read_csv(r'Datasets/Provided_Datasets/used-car-dataset-challenge/Car_Reviews.csv')
print(carReviewsDF.head())

   Audi  A1  5
0  audi  A3  4
1  AUDI  a4  2
2  audi  A5  5
3  audi  A6  3
4  Audi  A7  3


# Preparing and merging ratings using same steps

In [17]:
# Adding the column titles to the ratings data and saving
column_titles = ['brand', 'model', 'rating']
ratingsDF = pd.read_csv(r'Datasets/Provided_Datasets/used-car-dataset-challenge/Car_Reviews.csv', header=None, names=column_titles)

ratingsDF.to_csv('./Datasets/Processed_Data/Car_Reviews_Sanitized.csv', index=False, header=True)

In [18]:
ratingsDF.head()

,brand,model,rating
0,Audi,A1,5
1,audi,A3,4
2,AUDI,a4,2
3,audi,A5,5
4,audi,A6,3


In [19]:
# Convert 'brand' column to lowercase and replace 'hyundi' with 'hyundai'. 
ratingsDF['brand'] = ratingsDF['brand'].str.lower().replace(r'hyundi', 'hyundai', regex=True)
# Also converting 'volkswagen' to 'vw' to match existing data.
ratingsDF['brand'] = ratingsDF['brand'].replace(r'volkswagen', 'vw', regex=True)

# Strip leading/trailing whitespace and convert 'Model' column to lowercase. Title gives first initial a capital
ratingsDF['model'] = ratingsDF['model'].str.title()


# making all columns lowercase for future merger
ratingsDF.rename(columns={'brand': 'brand'.lower()}, inplace=True)
ratingsDF.rename(columns={'model': 'model'.lower()}, inplace=True)

ratingsDF.to_csv('./Datasets/Processed_Data/Car_Reviews_Sanitized.csv', index=False, header=True)

In [20]:
ratingsDF.head()

,brand,model,rating
0,audi,A1,5
1,audi,A3,4
2,audi,A4,2
3,audi,A5,5
4,audi,A6,3


# finding abnormal values here

In [21]:
# Finding and removing incorrect brand name

unique_brands_ratings = ratingsDF['brand'].unique()
print(f'ratings DF has {len(unique_brands_ratings)} unique entries.')
print(unique_brands_ratings)

indices_to_drop = ratingsDF[ratingsDF['brand'] == 'make'].index
ratingsDF.drop(indices_to_drop, inplace=True)

unique_brands_ratings = ratingsDF['brand'].unique()
print(f'ratings DF has {len(unique_brands_ratings)} unique entries.')
print(unique_brands_ratings)

ratings DF has 10 unique entries.
['audi' 'bmw' 'ford' 'hyundai' 'make' 'mercedes' 'skoda' 'toyota'
 'vauxhall' 'vw']
ratings DF has 9 unique entries.
['audi' 'bmw' 'ford' 'hyundai' 'mercedes' 'skoda' 'toyota' 'vauxhall' 'vw']


In [22]:
combined_cars_df = pd.read_csv('./Datasets/Processed_Data/used_car_data_combined.csv')
ratingsDF = pd.read_csv('Datasets/Processed_Data/Car_Reviews_Sanitized.csv')


# Strip leading/trailing whitespace and convert 'Model' column to lowercase
ratingsDF['model'] = ratingsDF['model'].str.lower()
ratingsDF['model'] = ratingsDF['model'].str.title()
ratingsDF['model'] = ratingsDF['model'].str.strip()

# Create a list of unique models from the combined_cars dataset for matching
#model_choices = combined_cars_df['model'].unique()
#print(model_choices)

#rating_choices = ratingsDF['model'].unique()
#print(rating_choices)

# Function to get the best match for a given model
def get_best_match(model, choices):
    
    if (model.lower() == 'ka') or (model.lower() == 'Ka'):
        return ' KA'
    
    else:
        match = process.extractOne(model, choices, scorer=process.fuzz.token_sort_ratio, score_cutoff=75)
        # If a match is found with a score above the cutoff, return the matched model, else return the original
        return match[0] if match else model


# Replace 'Model' entries in df with the best match from combined_cars_df
ratingsDF['model'] = ratingsDF['model'].apply(lambda x: get_best_match(x, model_choices))
ratingsDF['model'] = ratingsDF['model'].str.lstrip()

indices_to_drop = ratingsDF[ratingsDF['model'] == 'Model'].index
ratingsDF.drop(indices_to_drop, inplace=True)
#rating_choices = ratingsDF['model'].unique()
#print(rating_choices)

ratingsDF.to_csv('./Datasets/Processed_Data/Car_Reviews_Sanitized.csv', index=False, header=True)

In [23]:
unique_models_ratings = ratingsDF['model'].unique()
print(f'ratings DF has {len(unique_models_ratings)} unique model entries.')
#print(unique_models_ratings)


unique_models_combined = combined_cars_df['model'].unique()
print(f'Combined dataset DF has {len(unique_models_combined)} unique model entries.')
#print(unique_models_combined)


unique_models_existing_set = set(unique_models_combined)
unique_models_ratings_set = set(unique_models_ratings)

unique_to_existing_only = unique_models_existing_set - unique_models_ratings_set
print("Entries only present in existing files: ")
print(unique_to_existing_only)

unique_to_ratings_only = unique_models_ratings_set - unique_models_existing_set
print("Entries only present in ratings files: ")
print(unique_to_ratings_only)

ratings DF has 195 unique model entries.
Combined dataset DF has 195 unique model entries.
Entries only present in existing files: 
set()
Entries only present in ratings files: 
set()


In [24]:
print(ratingsDF.head())

  brand model  rating
0  audi    A1       5
1  audi    A3       4
2  audi    A4       2
3  audi    A5       5
4  audi    A6       3


In [25]:
combinedDF = pd.merge(combinedDF, ratingsDF[['brand', 'model', 'rating']],
                     on=['brand', 'model'],
                     how='left')

combinedDF.to_csv('./Datasets/Processed_Data/used_car_data_combined.csv', index=False, header=True)

In [26]:
combinedDF.head()

,brand,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,MaintenanceCostYearly,rating
0,audi,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4,787.98,5
1,audi,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0,661.97,3
2,audi,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4,708.90,5
3,audi,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0,595.38,2
4,audi,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0,500.10,4


# Data prep. complete